In [1]:
import sys
sys.path.append("src")

from core.populationNode import PopulationNode
from core.ops import _as_node, _broadcast_to_match, add, sub, mul, sum_pop, matvec
from core.parameter import Parameter
from models.activations import tanh
from core.optim import GD, Momentum


In [2]:
x = PopulationNode([4.5])
y1 = x.tanh()
y2 = y1.tanh()
y3 = y2.tanh()
y4 = y3.tanh()
y5 = y4.tanh()
y6 = y5.tanh()
y7 = y6.tanh()
y8 = y7.tanh()
y8.backprop(debug=True)

[NODE] op=tanh, value=[0.4134608463957053], grad=[1.0] | <-- Parents=[[0.43977852653283767]]
[NODE] op=tanh, value=[0.43977852653283767], grad=[0.829050128497747] | <-- Parents=[[0.4719561929220392]]
[NODE] op=tanh, value=[0.4719561929220392], grad=[0.6687075620489031] | <-- Parents=[[0.5125841451937511]]
[NODE] op=tanh, value=[0.5125841451937511], grad=[0.519757868915453] | <-- Parents=[[0.5662285757646344]]
[NODE] op=tanh, value=[0.5662285757646344], grad=[0.3831953839732797] | <-- Parents=[[0.6419540521719098]]
[NODE] op=tanh, value=[0.6419540521719098], grad=[0.26033727257499406] | <-- Parents=[[0.7614904913621627]]
[NODE] op=tanh, value=[0.7614904913621627], grad=[0.15305097953277294] | <-- Parents=[[0.9997532108480275]]
[NODE] op=tanh, value=[0.9997532108480275], grad=[0.06430164957431488] | <-- Parents=[[4.5]]
[NODE] op=leaf, value=[4.5], grad=[3.1733982853148134e-05] | <-- Parents=[]


In [3]:
xs = [-6, -3.0, -0.5, 0, .5, 3.0, 6]
x = PopulationNode(xs)
print("x: ", x)
y = x.tanh()

sum_pop(y).backprop(debug=True)

x:  PopulationNode( data=[-6.0, -3.0, -0.5, 0.0, 0.5, 3.0, 6.0], grad=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], op='leaf')
[NODE] op=sum, value=[0.0], grad=[1.0] | <-- Parents=[[-0.9999877116507956, -0.9950547536867305, -0.46211715726000974, 0.0, 0.46211715726000974, 0.9950547536867305, 0.9999877116507956]]
[NODE] op=tanh, value=[-0.9999877116507956, -0.9950547536867305, -0.46211715726000974, 0.0, 0.46211715726000974, 0.9950547536867305, 0.9999877116507956], grad=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] | <-- Parents=[[-6.0, -3.0, -0.5, 0.0, 0.5, 3.0, 6.0]]
[NODE] op=leaf, value=[-6.0, -3.0, -0.5, 0.0, 0.5, 3.0, 6.0], grad=[2.4576547405286142e-05, 0.009866037165440211, 0.7864477329659274, 1.0, 0.7864477329659274, 0.009866037165440211, 2.4576547405286142e-05] | <-- Parents=[]


In [4]:
x_scalar = PopulationNode(0.5)
y_scalar = x_scalar.tanh()
y_scalar.backprop()
print("sum of unit gradient", sum(x_scalar.grad))

sum of unit gradient 0.7864477329659274


In [5]:
for v in [-3, -1, 0, 1, 3]:
    x = PopulationNode(v)
    y = x.tanh()
    y.backprop()
    print(v, x.grad)

-3 [0.009866037165440211]
-1 [0.41997434161402614]
0 [1.0]
1 [0.41997434161402614]
3 [0.009866037165440211]


In [6]:


w = Parameter([5.0, 5.0, 5.0])
opt = GD([w], lr=0.1)

def pop_loss(w):
    return sum((wi - 2.0) ** 2 for wi in w.data)

history = []

for _ in range(15):
    L = pop_loss(w)
    for i in range(len(w.data)):
        w.grad[i] = 2 * (w.data[i] - 2.0)
    opt.step()
    history.append(w.data.copy())
    opt.zero_grad()

history

[[4.4, 4.4, 4.4],
 [3.9200000000000004, 3.9200000000000004, 3.9200000000000004],
 [3.5360000000000005, 3.5360000000000005, 3.5360000000000005],
 [3.2288000000000006, 3.2288000000000006, 3.2288000000000006],
 [2.9830400000000004, 2.9830400000000004, 2.9830400000000004],
 [2.7864320000000005, 2.7864320000000005, 2.7864320000000005],
 [2.6291456, 2.6291456, 2.6291456],
 [2.50331648, 2.50331648, 2.50331648],
 [2.402653184, 2.402653184, 2.402653184],
 [2.3221225472, 2.3221225472, 2.3221225472],
 [2.25769803776, 2.25769803776, 2.25769803776],
 [2.206158430208, 2.206158430208, 2.206158430208],
 [2.1649267441664, 2.1649267441664, 2.1649267441664],
 [2.13194139533312, 2.13194139533312, 2.13194139533312],
 [2.105553116266496, 2.105553116266496, 2.105553116266496]]

In [7]:
w = Parameter([5.0, 5.1, 4.9])
opt = GD([w], lr=0.1)

history_asym = []

for _ in range(15):
    for i in range(len(w.data)):
        w.grad[i] = 2 * (w.data[i] - 2.0)
    opt.step()
    history_asym.append(w.data.copy())
    opt.zero_grad()

history_asym

[[4.4, 4.4799999999999995, 4.32],
 [3.9200000000000004, 3.9839999999999995, 3.8560000000000003],
 [3.5360000000000005, 3.5871999999999997, 3.4848000000000003],
 [3.2288000000000006, 3.2697599999999998, 3.1878400000000005],
 [2.9830400000000004, 3.015808, 2.9502720000000004],
 [2.7864320000000005, 2.8126463999999998, 2.7602176000000003],
 [2.6291456, 2.65011712, 2.6081740800000004],
 [2.50331648, 2.520093696, 2.486539264],
 [2.402653184, 2.4160749568, 2.3892314112],
 [2.3221225472, 2.33285996544, 2.31138512896],
 [2.25769803776, 2.266287972352, 2.249108103168],
 [2.206158430208, 2.2130303778816, 2.1992864825344],
 [2.1649267441664, 2.17042430230528, 2.15942918602752],
 [2.13194139533312, 2.136339441844224, 2.127543348822016],
 [2.105553116266496, 2.109071553475379, 2.1020346790576125]]

In [8]:
import numpy as np

def _almost_equal_list(a, b, tol=1e-6):
    assert len(a) == len(b)
    for x, y in zip(a, b):
        assert abs(x - y) < tol


def test_matvec_forward_backward_diagonal():
    A = [[2.0, 0.0],
         [0.0, 3.0]]

    x = PopulationNode([1.0, 1.0], requires_grad=True)
    y = matvec(A, x)          # [2,3]
    loss = sum_pop(y)         # 5

    loss.zero_grad_graph()
    loss.backprop(debug=True)

    assert y.data == [2.0, 3.0]
    # dL/dx = A^T @ [1,1] = [2,3]
    _almost_equal_list(x.grad, [2.0, 3.0])


def test_matvec_forward_backward_random():
    rng = np.random.default_rng(0)
    A = rng.normal(size=(3, 2))
    x0 = rng.normal(size=(2,))

    x = PopulationNode(x0.tolist(), requires_grad=True)
    y = matvec(A, x)
    loss = sum_pop(y)

    loss.zero_grad_graph()
    loss.backprop()

    # analytic: A^T @ ones(3)
    expected = A.T @ np.ones(3)
    got = np.array(x.grad)
    assert np.max(np.abs(got - expected)) < 1e-6


In [9]:
test_matvec_forward_backward_diagonal()

[NODE] op=sum, value=[5.0], grad=[1.0] | <-- Parents=[[2.0, 3.0]]
[NODE] op=matvec, value=[2.0, 3.0], grad=[1.0, 1.0] | <-- Parents=[[1.0, 1.0]]
[NODE] op=leaf, value=[1.0, 1.0], grad=[2.0, 3.0] | <-- Parents=[]


In [10]:
test_matvec_forward_backward_random()

In [11]:
def close(a, b, tol=1e-9):
    return all(abs(x - y) < tol for x, y in zip(a, b))

def test_momentum_beta_zero_equals_gd_update():
    """
    If beta = 0, then:
        v <- -lr * grad
        w <- w + v = w - lr * grad
    So it should behave like plain GD for that step.
    """
    w = Parameter([1.0, 2.0])
    w.grad = [1.0, 1.0]

    opt = Momentum([w], lr=0.1, beta=0.0)
    opt.step()

    assert close(w.data, [0.9, 1.9])


def test_momentum_two_steps_matches_hand_computation():
    """
    Hand-check 2 steps with constant gradient to verify velocity accumulation.

    Let w0 = 1.0, grad = 1.0, lr = 0.1, beta = 0.9
    v0 = 0

    Step 1:
      v1 = 0.9*v0 - 0.1*1 = -0.1
      w1 = w0 + v1 = 0.9

    Step 2:
      v2 = 0.9*v1 - 0.1*1 = 0.9*(-0.1) - 0.1 = -0.19
      w2 = w1 + v2 = 0.9 - 0.19 = 0.71
    """
    w = Parameter([1.0])
    opt = Momentum([w], lr=0.1, beta=0.9)

    # Step 1
    w.grad = [1.0]
    opt.step()
    assert close(w.data, [0.9])

    # Step 2 (same grad)
    w.grad = [1.0]
    opt.step()
    assert close(w.data, [0.71])


def test_momentum_zero_grad_clears_param_grads_only():
    """
    zero_grad should clear Parameter.grad.
    It should NOT reset velocity unless you explicitly add such behavior.
    """
    w = Parameter([1.0])
    opt = Momentum([w], lr=0.1, beta=0.9)

    w.grad = [1.0]
    opt.step()  # now velocity is non-zero

    w.grad = [5.0]
    opt.zero_grad()
    assert w.grad == [0.0]


if __name__ == "__main__":
    # Run without pytest
    test_momentum_beta_zero_equals_gd_update()
    test_momentum_two_steps_matches_hand_computation()
    test_momentum_zero_grad_clears_param_grads_only()
    print("All Momentum tests passed.")

All Momentum tests passed.


In [12]:

def header(title):
    print("\n" + "=" * 60)
    print(title)
    print("=" * 60)

def close(a, b, tol=1e-9):
    return all(abs(x - y) < tol for x, y in zip(a, b))

# ------------------------------------------------------------
# Test 1: Scalar test
# ------------------------------------------------------------
header("Test 1: Scalar Test (Simple Chain Rule)")

x = PopulationNode(2.0)
y = PopulationNode(3.0)
z = x * y + x
print(f"z =  (expected [8.0]): {z.data}")
z.backprop()

print(f"x.grad (expected [4.0]): {x.grad}")
print(f"y.grad (expected [2.0]): {y.grad}")

assert close(x.grad, [4.0]), "Incorrect gradient for x"
assert close(y.grad, [2.0]), "Incorrect gradient for y"

print("✓ Passed: gradients match analytical derivatives")

# ------------------------------------------------------------
# Test 2: Vector test
# ------------------------------------------------------------
header("Test 2: Vector Test (Vector Chain Rule)")

x = PopulationNode([1.0, 2.0])
y = PopulationNode([3.0, 4.0])
z = sum_pop(x + y)
z.backprop()

print(f"x.grad (expected [1.0, 1.0]): {x.grad}")
print(f"y.grad (expected [1.0, 1.0]): {y.grad}")

assert close(x.grad, [1.0, 1.0]), "Incorrect gradient for x"
assert close(y.grad, [1.0, 1.0]), "Incorrect gradient for y"

print("✓ Passed: vector gradients match analytical derivatives")

# ------------------------------------------------------------
# Test 3: Elementwise multiply + sum reduction + tanh neuron
# ------------------------------------------------------------
header("Test 3: (w * x) -> sum -> + b -> tanh")

w = PopulationNode([1.0, -1.0])
x = PopulationNode([1.0, 0.0])
b = PopulationNode(0.0)

z = sum_pop(w * x) + b
a = z.tanh()
a.backprop()

dadz = 1.0 - a.data[0] ** 2
expected_w = [dadz * x.data[0], dadz * x.data[1]]
expected_x = [dadz * w.data[0], dadz * w.data[1]]
expected_b = [dadz]

print(f"a.data: {a.data}")
print(f"w.grad: {w.grad} (expected {expected_w})")
print(f"x.grad: {x.grad} (expected {expected_x})")
print(f"b.grad: {b.grad} (expected {expected_b})")

assert close(w.grad, expected_w), "Incorrect gradient for weights"
assert close(x.grad, expected_x), "Incorrect gradient for inputs"
assert close(b.grad, expected_b), "Incorrect gradient for bias"

print("✓ Passed: tanh derivative and neuron gradients are correct")





Test 1: Scalar Test (Simple Chain Rule)
z =  (expected [8.0]): [8.0]
x.grad (expected [4.0]): [4.0]
y.grad (expected [2.0]): [2.0]
✓ Passed: gradients match analytical derivatives

Test 2: Vector Test (Vector Chain Rule)
x.grad (expected [1.0, 1.0]): [1.0, 1.0]
y.grad (expected [1.0, 1.0]): [1.0, 1.0]
✓ Passed: vector gradients match analytical derivatives

Test 3: (w * x) -> sum -> + b -> tanh
a.data: [0.7615941559557649]
w.grad: [0.41997434161402614, 0.0] (expected [0.41997434161402614, 0.0])
x.grad: [0.41997434161402614, -0.41997434161402614] (expected [0.41997434161402614, -0.41997434161402614])
b.grad: [0.41997434161402614] (expected [0.41997434161402614])
✓ Passed: tanh derivative and neuron gradients are correct


In [13]:
# ------------------------------------------------------------
# Test 4: Gradient accumulation
# ------------------------------------------------------------
header("Test 4: Gradient accumulation (no zero_grad)")

x = PopulationNode(2.0)
y = PopulationNode(3.0)
z = x * y + x
print("Z: ",z)
print("-----------Before backprop-------------")
print("pre-GZ: ",z.grad)
print("pre-Gy: ",y.grad)
print("pre-Gx: ",x.grad)
z.backprop()
print("-----------After second backprop-------------")
print("Z: ",z)
print("pre-GZ: ",z.grad)
print("pre-Gy: ",y.grad)
print("pre-Gx: ",x.grad)

g1x, g1y = x.grad[:], y.grad[:]
print("pre-g1x: ", g1x, "pre-g2y: ", g1y)
print("-----------Before second backprop-------------")
print("pre-GZ: ",z.grad)
print("pre-Gy: ",y.grad)
print("pre-Gx: ",x.grad)
z.backprop()  # no reset
print("-----------After second backprop-------------")

print("pre-GZ: ",z.grad)
print("pre-Gy: ",y.grad)
print("pre-Gx: ",x.grad)
g2x, g2y = x.grad[:], y.grad[:]
print("pre-g1x: ", g2x, "pre-g2y: ", g2y)
print("\n ----Checking------")
print("g2x: ", g2x, "g2y: ", g2y)
print(f"First x.grad: {g1x}, second x.grad: {g2x}")
print(f"First y.grad: {g1y}, second y.grad: {g2y}")

assert close(g2x, [2 * g1x[0]]), "Gradients should accumulate for x"
assert close(g2y, [2 * g1y[0]]), "Gradients should accumulate for y"
print("✓ Passed: gradients accumulate as expected")

print("\nAll tests passed successfully.")


Test 4: Gradient accumulation (no zero_grad)
Z:  PopulationNode( data=[8.0], grad=[0.0], op='+')
-----------Before backprop-------------
pre-GZ:  [0.0]
pre-Gy:  [0.0]
pre-Gx:  [0.0]
-----------After second backprop-------------
Z:  PopulationNode( data=[8.0], grad=[1.0], op='+')
pre-GZ:  [1.0]
pre-Gy:  [2.0]
pre-Gx:  [4.0]
pre-g1x:  [4.0] pre-g2y:  [2.0]
-----------Before second backprop-------------
pre-GZ:  [1.0]
pre-Gy:  [2.0]
pre-Gx:  [4.0]
-----------After second backprop-------------
pre-GZ:  [1.0]
pre-Gy:  [4.0]
pre-Gx:  [8.0]
pre-g1x:  [8.0] pre-g2y:  [4.0]

 ----Checking------
g2x:  [8.0] g2y:  [4.0]
First x.grad: [4.0], second x.grad: [8.0]
First y.grad: [2.0], second y.grad: [4.0]
✓ Passed: gradients accumulate as expected

All tests passed successfully.


In [14]:

w1 = Parameter([1.0])
w2 = Parameter([2.0])

w1.grad = [1.0]
w2.grad = [2.0]

opt = GD([w1, w2], lr=0.1)
opt.step()

assert w1.data == [0.9]
assert w2.data == [1.8]

In [15]:
opt.zero_grad()
assert w1.grad == [0.0]
assert w2.grad == [0.0]

In [16]:


def test_grad_accumulates_then_resets():
    x = PopulationNode([1.0, 2.0], requires_grad=True)
    print(x)
    loss = sum_pop(x)  # loss = x1 + x2
    print(x)
    print("loss: ",loss)
    loss.zero_grad_graph()
    print("loss: ",loss)
    loss.backprop(debug=True)
    print("loss: ",loss)
    g1 = list(x.grad)
    print(g1)
    assert g1 == [1.0, 1.0]
    print("✓ Passed: gradients reset as expected")

    # backprop again without clearing => accumulates
    loss.backprop(debug=True)
    g2 = list(x.grad)
    assert g2 == [2.0, 2.0]
    print("✓ Passed: gradients accumulate as expected")

    # clear entire graph
    loss.zero_grad_graph()
    assert x.grad == [0.0, 0.0]
    print("✓ Passed: clear entire graph's gradient as expected")

    print("\nAll tests passed successfully.")


In [17]:
test_grad_accumulates_then_resets()

PopulationNode( data=[1.0, 2.0], grad=[0.0, 0.0], op='leaf')
PopulationNode( data=[1.0, 2.0], grad=[0.0, 0.0], op='leaf')
loss:  PopulationNode( data=[3.0], grad=[0.0], op='sum')
loss:  PopulationNode( data=[3.0], grad=[0.0], op='sum')
[NODE] op=sum, value=[3.0], grad=[1.0] | <-- Parents=[[1.0, 2.0]]
[NODE] op=leaf, value=[1.0, 2.0], grad=[1.0, 1.0] | <-- Parents=[]
loss:  PopulationNode( data=[3.0], grad=[1.0], op='sum')
[1.0, 1.0]
✓ Passed: gradients reset as expected
[NODE] op=sum, value=[3.0], grad=[1.0] | <-- Parents=[[1.0, 2.0]]
[NODE] op=leaf, value=[1.0, 2.0], grad=[2.0, 2.0] | <-- Parents=[]
✓ Passed: gradients accumulate as expected
✓ Passed: clear entire graph's gradient as expected

All tests passed successfully.


In [18]:
def test_population_autodiff_engine():
    # ------------------------------------------------------------
    # 1. Leaf creation & shape
    # ------------------------------------------------------------
    x = PopulationNode([1.0, 2.0], requires_grad=True)
    y = PopulationNode([3.0, 4.0], requires_grad=True)
    assert x.data == [1.0, 2.0]
    assert y.data == [3.0, 4.0]

    # ------------------------------------------------------------
    # 2. Elementwise ops: add, sub, mul
    # ------------------------------------------------------------
    z1 = x + y
    assert z1.data == [4.0, 6.0]

    z2 = x - y
    assert z2.data == [-2.0, -2.0]

    z3 = x * y
    assert z3.data == [3.0, 8.0]

    # ------------------------------------------------------------
    # 3. Nonlinearity: tanh
    # ------------------------------------------------------------
    t = x.tanh()
    import math
    assert t.data == [math.tanh(1.0), math.tanh(2.0)]

    # ------------------------------------------------------------
    # 4. Reduction: sum
    # ------------------------------------------------------------
    s = sum_pop(x)
    assert s.data == [3.0]

    # ------------------------------------------------------------
    # 5. Backprop through simple graph
    #    s = x.sum() => ds/dx = [1, 1]
    # ------------------------------------------------------------
    x.zero_grad()
    s.backprop()
    assert x.grad == [1.0, 1.0]

    # ------------------------------------------------------------
    # 6. Multi-path graph:
    #    z = x*y + x
    #    dz/dx = y + 1
    #    dz/dy = x
    # ------------------------------------------------------------
    x = PopulationNode([2.0], requires_grad=True)
    y = PopulationNode([3.0], requires_grad=True)
    z = x * y + x

    x.zero_grad()
    y.zero_grad()
    z.backprop()

    assert x.grad == [4.0]   # y + 1 = 3 + 1
    assert y.grad == [2.0]   # x = 2

    # ------------------------------------------------------------
    # 7. Gradient accumulation on leaf nodes
    # ------------------------------------------------------------
    z.backprop()  # second backward, no zero_grad

    assert x.grad == [8.0]   # 4 + 4
    assert y.grad == [4.0]   # 2 + 2

    # ------------------------------------------------------------
    # 8. Intermediate nodes must NOT accumulate
    #    (We check this indirectly by verifying correct doubling)
    # ------------------------------------------------------------
    # If intermediate nodes accumulated, x.grad would be wrong (11, 12, etc.)
    assert x.grad == [8.0]
    assert y.grad == [4.0]

    # ------------------------------------------------------------
    # 9. zero_grad_graph clears entire graph
    # ------------------------------------------------------------
    z.zero_grad_graph()
    assert x.grad == [0.0]
    assert y.grad == [0.0]
    # intermediate nodes should also be zeroed (implicitly tested)

    # ------------------------------------------------------------
    # 10. Vector-output backward
    # ------------------------------------------------------------
    v = PopulationNode([1.0, 2.0, 3.0], requires_grad=True)
    out = v * 2.0  # elementwise
    v.zero_grad()
    out.backprop(seed_grad=[1.0, 1.0, 1.0])
    assert v.grad == [2.0, 2.0, 2.0]
    print("✓ Full engine test passed successfully.")


In [19]:
test_population_autodiff_engine()

✓ Full engine test passed successfully.


In [20]:
# engine_tests.py
import math
import sys


# ---------------------------------------------------------------------
# Helpers
# ---------------------------------------------------------------------
ATOL = 1e-6
RTOL = 1e-6
FD_EPS = 1e-5


def assert_close(a, b, atol=ATOL, rtol=RTOL, msg=""):
    if not math.isclose(float(a), float(b), abs_tol=atol, rel_tol=rtol):
        raise AssertionError(msg or f"Expected {b}, got {a}")


def assert_close_list(a, b, atol=ATOL, rtol=RTOL, msg=""):
    if len(a) != len(b):
        raise AssertionError(msg or f"Length mismatch: {len(a)} vs {len(b)}")
    for i, (ai, bi) in enumerate(zip(a, b)):
        if not math.isclose(float(ai), float(bi), abs_tol=atol, rel_tol=rtol):
            raise AssertionError(msg or f"Mismatch at idx={i}: got {ai}, expected {bi}")


def expect_raises(exc_type, fn, *args, **kwargs):
    try:
        fn(*args, **kwargs)
    except exc_type:
        return
    except Exception as e:
        raise AssertionError(f"Expected {exc_type.__name__}, got {type(e).__name__}: {e}") from e
    raise AssertionError(f"Expected {exc_type.__name__} but no exception was raised")


def scalar_objective_from_fn(fn, x_vals):
    """
    Build a graph for L(x) and return (x_node, loss_node).
    If fn returns a vector, we reduce with sum_pop to scalar.
    """
    x = PopulationNode(list(x_vals))
    out = fn(x)
    if len(out.data) != 1:
        out = sum_pop(out)
    return x, out


def finite_diff_grad(fn, x0, eps=FD_EPS):
    """
    Central difference gradient for scalar objective L(x).
    """
    g = [0.0] * len(x0)
    for i in range(len(x0)):
        xp = list(x0)
        xm = list(x0)
        xp[i] += eps
        xm[i] -= eps

        _, out_p = scalar_objective_from_fn(fn, xp)
        _, out_m = scalar_objective_from_fn(fn, xm)

        Lp = out_p.data[0]
        Lm = out_m.data[0]
        g[i] = (Lp - Lm) / (2 * eps)
    return g


def engine_grad(fn, x0):
    x, out = scalar_objective_from_fn(fn, x0)
    out.backprop()
    return list(x.grad), out

# ---------------------------------------------------------------------
# Shared‑node correctness (multi‑path gradient flow)
# ---------------------------------------------------------------------

def test_shared_node_simple():
    """
    z = m + m where m = x * y
    Ensures correct accumulation on shared subgraphs.
    """
    x = PopulationNode([2.0], requires_grad=True)
    y = PopulationNode([3.0], requires_grad=True)

    m = x * y
    z = m + m   # shared node

    x.zero_grad()
    y.zero_grad()
    z.backprop()

    # dz/dx = 2*y = 6
    # dz/dy = 2*x = 4
    assert x.grad == [6.0]
    assert y.grad == [4.0]


def test_shared_node_deep():
    """
    z = m + (m * 2) where m = x * y
    Ensures correct multi‑path gradient flow.
    """
    x = PopulationNode([2.0], requires_grad=True)
    y = PopulationNode([5.0], requires_grad=True)

    m = x * y
    z = m + (m * 2.0)   # 3*m total

    x.zero_grad()
    y.zero_grad()
    z.backprop()

    # dz/dx = 3*y = 15
    # dz/dy = 3*x = 6
    assert x.grad == [15.0]
    assert y.grad == [6.0]


def test_shared_node_multiple_backward():
    """
    Ensures:
    - leaf nodes accumulate across backward calls
    - shared intermediate nodes DO NOT accumulate across calls
    """
    x = PopulationNode([2.0], requires_grad=True)
    y = PopulationNode([3.0], requires_grad=True)

    m = x * y
    z = m + m

    x.zero_grad()
    y.zero_grad()

    # First backward
    z.backprop()
    g1x, g1y = x.grad[:], y.grad[:]

    # Second backward (no zero_grad)
    z.backprop()
    g2x, g2y = x.grad[:], y.grad[:]

    # Leaves accumulate
    assert g2x == [2 * g1x[0]]
    assert g2y == [2 * g1y[0]]

    # Intermediate node m should NOT accumulate across calls
    # (implicitly tested because if it did, leaf grads would be wrong)
    assert g2x == [12.0]   # 6 → 12
    assert g2y == [8.0]    # 4 → 8


def test_shared_node_vector():
    """
    Shared node with vector outputs.
    v = [1,2,3]
    m = v * 2
    z = m + m
    dz/dv = 4
    """
    v = PopulationNode([1.0, 2.0, 3.0], requires_grad=True)

    m = v * 2.0
    z = m + m

    v.zero_grad()
    z.backprop()

    assert v.grad == [4.0, 4.0, 4.0]

# ---------------------------------------------------------------------
# Tests
# ---------------------------------------------------------------------
def test_forward_ops():
    x = PopulationNode([1.0, 2.0, 3.0])
    y = PopulationNode([10.0, 20.0, 30.0])

    assert x.data == [1.0, 2.0, 3.0]
    assert (x + y).data == [11.0, 22.0, 33.0]
    assert (x * y).data == [10.0, 40.0, 90.0]
    assert (y - x).data == [9.0, 18.0, 27.0]

    t = x.tanh().data
    assert all(-1.0 < v < 1.0 for v in t)

    s = sum_pop(x)
    assert s.data == [6.0]


def test_matvec_forward():
    A = [[1.0, 2.0], [3.0, 4.0]]
    x = PopulationNode([5.0, 6.0])
    y = matvec(A, x)
    assert y.data == [17.0, 39.0]


def test_backward_scalar_graph():
    # z = x*y + x, x=2, y=3 => dz/dx = 4, dz/dy = 2
    x = PopulationNode(2.0)
    y = PopulationNode(3.0)
    z = x * y + x
    z.backprop()
    assert_close_list(x.grad, [4.0])
    assert_close_list(y.grad, [2.0])
    assert_close_list(z.grad, [1.0])


def test_backward_vector_graph():
    # L = sum(x*y + x) for x=[1,2,3], y=[4,5,6]
    # dL/dx = y + 1 = [5,6,7], dL/dy = x = [1,2,3]
    x = PopulationNode([1.0, 2.0, 3.0])
    y = PopulationNode([4.0, 5.0, 6.0])
    L = sum_pop(x * y + x)
    L.backprop()

    assert_close_list(x.grad, [5.0, 6.0, 7.0])
    assert_close_list(y.grad, [1.0, 2.0, 3.0])


def test_requires_grad_false_constant():
    x = PopulationNode([1.0, 2.0, 3.0])
    c = PopulationNode([10.0, 10.0, 10.0], requires_grad=False)
    L = sum_pop(x * c)
    L.backprop()

    assert_close_list(x.grad, [10.0, 10.0, 10.0])
    assert_close_list(c.grad, [0.0, 0.0, 0.0])


def test_broadcast_add_backward():
    # out = sum(x + v) where x is scalar => dx = len(v), dv_i = 1
    x = PopulationNode(2.0)
    v = PopulationNode([1.0, 2.0, 3.0])
    out = sum_pop(x + v)
    out.backprop()

    assert_close_list(x.grad, [3.0])
    assert_close_list(v.grad, [1.0, 1.0, 1.0])


def test_broadcast_mul_backward():
    # out = sum(x * v) where x scalar => dx = sum(v), dv_i = x
    x = PopulationNode(2.0)
    v = PopulationNode([1.0, 2.0, 3.0])
    out = sum_pop(x * v)
    out.backprop()

    assert_close_list(x.grad, [6.0])
    assert_close_list(v.grad, [2.0, 2.0, 2.0])


def test_shape_mismatch_errors():
    a = PopulationNode([1.0, 2.0])
    b = PopulationNode([1.0, 2.0, 3.0])
    expect_raises(ValueError, lambda: a + b)
    expect_raises(ValueError, lambda: a * b)

    A = [[1.0, 2.0], [3.0, 4.0]]
    x = PopulationNode([1.0, 2.0, 3.0])
    expect_raises(ValueError, lambda: matvec(A, x))


def test_zero_grad_graph_clears_all():
    x = PopulationNode(2.0)
    y = PopulationNode(3.0)
    z = x * y + x
    z.backprop()
    assert x.grad != [0.0] and y.grad != [0.0] and z.grad != [0.0]

    z.zero_grad_graph()
    assert_close_list(x.grad, [0.0])
    assert_close_list(y.grad, [0.0])
    assert_close_list(z.grad, [0.0])


def test_accumulation_semantics_leaves_accumulate_intermediates_reset():
    """
    This matches your "Approach A":
      - leaf grads accumulate across repeated backprop calls
      - intermediate grads should NOT accumulate (they should be reset inside backprop)
    """
    x = PopulationNode(2.0)
    y = PopulationNode(3.0)
    a = x * y          # intermediate
    z = a + x          # output

    z.backprop()
    g1x, g1y, g1a = x.grad[0], y.grad[0], a.grad[0]

    z.backprop()  # no manual reset
    g2x, g2y, g2a = x.grad[0], y.grad[0], a.grad[0]

    assert_close(g2x, 2 * g1x, msg="Leaf x.grad should accumulate (double).")
    assert_close(g2y, 2 * g1y, msg="Leaf y.grad should accumulate (double).")
    assert_close(g2a, g1a, msg="Intermediate a.grad should NOT accumulate under Approach A.")


def test_gradcheck_tanh_chain():
    # L = sum(tanh(x) * x)
    fn = lambda x: sum_pop(x.tanh() * x)
    x0 = [0.1, -0.7, 1.3]
    g_eng, _ = engine_grad(fn, x0)
    g_fd = finite_diff_grad(fn, x0)
    # tanh can be slightly sensitive; loosen tolerance a bit
    assert_close_list(g_eng, g_fd, atol=2e-4, rtol=2e-4)


def test_gradcheck_matvec():
    A = [
        [0.3, 1.7, -2.0],
        [1.0, -0.5, 0.2],
    ]
    fn = lambda x: sum_pop(matvec(A, x))
    x0 = [0.4, -0.8, 1.1]
    g_eng, _ = engine_grad(fn, x0)
    g_fd = finite_diff_grad(fn, x0)
    assert_close_list(g_eng, g_fd, atol=2e-4, rtol=2e-4)


def test_parameter_and_optimizers():
    # Parameter.step
    w = Parameter([1.0, -2.0, 0.5])
    x = PopulationNode([3.0, 4.0, 5.0], requires_grad=False)
    loss = sum_pop(w * x)
    loss.backprop()

    assert_close_list(w.grad, [3.0, 4.0, 5.0])
    old = w.data[:]
    lr = 0.1
    w.step(lr)
    assert_close_list(w.data, [old[i] - lr * w.grad[i] for i in range(3)])

    # GD optimizer + zero_grad
    w2 = Parameter([1.0, 2.0])
    x2 = PopulationNode([10.0, -1.0], requires_grad=False)
    loss2 = sum_pop(w2 * x2)
    loss2.backprop()
    opt = GD([w2], lr=0.5)
    opt.step()
    assert_close_list(w2.data, [1.0 - 0.5 * 10.0, 2.0 - 0.5 * (-1.0)])
    opt.zero_grad()
    assert_close_list(w2.grad, [0.0, 0.0])

    # Momentum reference math (deterministic)
    w3 = Parameter([1.0, 2.0])
    w3.grad = [3.0, -4.0]
    optm = Momentum([w3], lr=0.1, beta=0.9)

    # step1: v=[-0.3, 0.4], w=[0.7,2.4]
    optm.step()
    assert_close_list(w3.data, [0.7, 2.4], atol=1e-9, rtol=0.0)

    # step2: v=0.9*v - lr*grad => [-0.57, 0.76], w=[0.13,3.16]
    w3.grad = [3.0, -4.0]
    optm.step()
    assert_close_list(w3.data, [0.13, 3.16], atol=1e-9, rtol=0.0)


# ---------------------------------------------------------------------
# Minimal runner
# ---------------------------------------------------------------------
TESTS = [
    test_forward_ops,
    test_matvec_forward,
    test_backward_scalar_graph,
    test_backward_vector_graph,
    test_shared_node_simple,
    test_shared_node_deep,
    test_shared_node_multiple_backward,
    test_shared_node_vector,
    test_requires_grad_false_constant,
    test_broadcast_add_backward,
    test_broadcast_mul_backward,
    test_shape_mismatch_errors,
    test_zero_grad_graph_clears_all,
    test_accumulation_semantics_leaves_accumulate_intermediates_reset,
    test_gradcheck_tanh_chain,
    test_gradcheck_matvec,
    test_parameter_and_optimizers,
]


def run_all():
    passed = 0
    failed = 0

    print("=" * 72)
    print("Running engine tests (plain asserts, no pytest)")
    print("=" * 72)

    for t in TESTS:
        name = t.__name__
        try:
            t()
            print(f"[PASS] {name}")
            passed += 1
        except Exception as e:
            print(f"[FAIL] {name}: {e}")
            traceback.print_exc()
            failed += 1

    print("=" * 72)
    print(f"Summary: {passed} passed, {failed} failed, total {passed + failed}")
    print("=" * 72)

    return 0 if failed == 0 else 1


if __name__ == "__main__":
    sys.exit(run_all())



Running engine tests (plain asserts, no pytest)
[PASS] test_forward_ops
[PASS] test_matvec_forward
[PASS] test_backward_scalar_graph
[PASS] test_backward_vector_graph
[PASS] test_shared_node_simple
[PASS] test_shared_node_deep
[PASS] test_shared_node_multiple_backward
[PASS] test_shared_node_vector
[PASS] test_requires_grad_false_constant
[PASS] test_broadcast_add_backward
[PASS] test_broadcast_mul_backward
[PASS] test_shape_mismatch_errors
[PASS] test_zero_grad_graph_clears_all
[PASS] test_accumulation_semantics_leaves_accumulate_intermediates_reset
[PASS] test_gradcheck_tanh_chain
[PASS] test_gradcheck_matvec
[PASS] test_parameter_and_optimizers
Summary: 17 passed, 0 failed, total 17


SystemExit: 0

C:\Users\ordon\anaconda3\envs\autodiff\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
